here the readme with full info http://files.grouplens.org/datasets/movielens/ml-25m-README.html about this DF

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. https://doi.org/10.1145/2827872

In [2]:
# Cargo las librerias

In [3]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random
import pandas as pd
import hashlib

Cargo ambos datasets

In [5]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

Los Mergeo usando movieID

Descarto 'timestamp'

In [7]:
completo = pd.merge(ratings, movies, on='movieId', how='inner')
completo = completo.drop(['timestamp'], axis=1)

Paso las siguientes a int

In [9]:
completo['userId']=completo['userId'].astype('int')
completo['movieId']=completo['movieId'].astype('int')
completo['rating']=completo['rating'].astype('int')

Tomo un sample de 10000

In [11]:
completo = completo.sample(n = 10000)

Guardo el Pre-Procesamiento

In [12]:
completo.to_csv("sample.csv")

Armo diccionarios de similitud por pelicula y por usuario

In [14]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

itemNames = {}

for d in range(len(completo)):
    d_ = completo.iloc[d]
    user,item = d_['userId'], d_['movieId']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d_['title']

In [140]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [16]:
def mostSimilar(iD, n):
    similarities = []
    users = usersPerItem[iD]
    for i2 in usersPerItem:
        if i2 == iD: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:n]

In [17]:
# Reset index to use loc dataframe function

In [18]:
completo = completo.reset_index()

In [20]:
query = completo.loc[11]['movieId']
query

3893

Busco la pelicula similar a la del query

In [22]:
itemNames[query]

'Nurse Betty (2000)'

In [23]:
[itemNames[x[1]] for x in mostSimilar(query, 10)]

['Tangled (2010)',
 'The Good Liar (2019)',
 'Countdown (2019)',
 'Takumi - A 60,000 hour story on the survival of human craft. (2019)',
 'Walt Disney (2015)',
 'Breslin and Hamill: Deadline Artists (2018)',
 'Saint Maud (2019)',
 'Parasite (2019)',
 'Yesterday (2019)',
 'Toy Story 4 (2019)']

Calculo la media del puntaje

In [42]:
ratingMean = sum(completo['rating'])/len(completo)    
ratingMean

3.3838

In [186]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in range(len(completo)):
        d_ = completo.iloc[d]
        i2 = d_['movieId']
        if i2 == item:
            ratings.append(d_['rating'])
            similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)] 
        return sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

Intento predecir el rating de 2 pelis

In [187]:
user,item = completo.loc[10]['userId'], completo.loc[10]['movieId']
predictRating(user, item)

3.466666666666667

In [188]:
user,item = completo.loc[17]['userId'], completo.loc[17]['movieId']
predictRating(user, item)

3.923076923076923

Medium square error

In [189]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

Lista de ratings

In [191]:
rating_list=[]

rating_list = completo.groupby(['movieId'])["rating"].mean()

Armo una lista de predicción

In [192]:
predigo=[]
   
for i in range(0, len(completo)):
    user,item = completo.iloc[i]['userId'], completo.iloc[i]['movieId']
    predigo.append(predictRating(user, item))

In [193]:
labels = []

for i in range(0, len(completo)):
    labels.append(completo.iloc[i]['rating'])

In [194]:
print(MSE(rating_list, labels),MSE(predigo, labels))

2.3105369067642516 0.6263456874101259
